In [1]:
import numpy as np
import logging

# Create util functions

In [72]:
def to_sym_tensor(data:np.ndarray, dtype=np.float64) -> np.ndarray:
    
    # check if data is an array so that we can use "shape" value
    if not isinstance(data, np.ndarray):
        x = np.array(data, dtype=dtype)
    else:
        x = data
        
    # check if is a simple vectorizer tensor -> (6,)
    if len(x.shape) == 1:
        if not x.shape[0] == 6:
            raise ValueError("1D data must contain 6 elements -> [x, y, z, xy, yz, xz]")
        
        return np.asarray([
                    [x[0], x[3], x[5]],
                    [x[3], x[1], x[4]],
                    [x[5], x[4], x[2]]
                ], dtype=dtype)
    # if data is not a simple vectorizer tensor, 
    # it can either be in the correct shape
    # or it needs to be converted to tensor format   
    elif len(x.shape) == 2:
        # if data is already in correct shape -> [3,3]
        if x.shape[1] == 3:
            if not x.shape[0] == 3:
                raise ValueError("2D data with 3 elements on second axis must follow the shape (3,3)")
            else:
                return np.asarray(x, dtype=dtype)
        
        # if data is an array of vectorized tensors -> must have shape of [n,6]
        elif not x.shape[1] == 6:
            raise ValueError("2D data must have (n,6) shape -> [[x, y, z, xy, yz, xz], ...]")
        else:
            return np.asarray([to_sym_arr(d) for d in x], dtype=dtype)
    else:
        raise ValueError("Accepted shapes: (6,), (3,3) or (n,6)")

In [440]:
def apply_tensor_transformation(tensor, vector, dtype=np.float64, log_level=logging.WARN):
    
    # Expected input shapes:
    # tensor -> (3,3) or (n,3,3)
    # vector --> [(3) or (n,3)] or [(3,3) or (n,3,3)]
    #       note: if vector is (n, 3,3), it is technically another tensor
    
    # ----------------------------------------------------------------
    # set logger
    logging.basicConfig()
    logger = logging.getLogger('apply_tensor_vector_transformation')
    logger.setLevel(log_level)
    
    # ----------------------------------------------------------------
    # ensure inputs are numpy arrays
    # so that we can use "shape" value
    
    # tensor must be a numpy array
    if not isinstance(tensor, np.ndarray):
        x = np.array(tensor, dtype=dtype)
    else:
        x = tensor
    # vector must be a numpy array
    if not isinstance(vector, np.ndarray):
        v = np.array(vector, dtype=dtype)
    else:
        v = vector
    
    # ----------------------------------------------------------------
    # check shapes
    # since we accept mutiple shape input formats, we must establish
    # a consistent shape for efficient computation
    # Needed shapes:
    # tensor -> (n,3,3)
    # vector -> (n,1,3) or (n,3,3)
    
    # log received shapes
    logger.debug("Shapes received:")
    logger.debug("x shape: {}".format(x.shape))
    logger.debug("v shape: {}".format(v.shape))
    
    # ----- check for "short" input shapes
    # expand dimensions if single tensor: (3,3) -> (1,3,3)
    if len(x.shape) == 2:
        x = np.expand_dims(x, 0)
    # expand dimensions if single vector: (3) -> (1,3)
    if len(v.shape) == 1:
        v = np.expand_dims(v, 0)
    
    # ----- check for required shapes
    # check for expected vector shape -> must have 3 elements in last axis
    # we need to check here in order to perform proper "repeat" operation (see below)
    if not v.shape[1] == 3:
        raise ValueError("Vector must have 3 elements on last axis. Received: {}".format(vector.shape[1]))
    # check for expected tensor shape -> (n,3,3)
    if len(x.shape) != 3 or x.shape[1] != 3 or x.shape[2] != 3:
        raise ValueError("Invalid shape found: expected (n,3,3), received {}".format(x.shape))
    
    # ----- match tensor shapes for matrix multiplication
    # match v shape with x shape:
    if v.shape[0] != x.shape[0]:
        # if v == (1,3) then v -> (n,3)
        if v.shape[0] == 1:
            v = np.repeat(v, x.shape[0], axis=0)
        # if v == (m,3) and m != n, we cannot perform matrix multiplication
        else:
            raise ValueError("tensors must have same shape at first axis if reference is not a vector: x {}, v {}".format(x.shape[0], v.shape[0]))
    # match array shape for consistent matrix multiplication
    # if v == (n,3) then v -> (n,1,3)
    if len(v.shape) == 2:
        v = np.expand_dims(v, axis=1)
    
    # log vectors after correction
    logger.debug("Shapes vector shape correction:")
    logger.debug("x shape: {}".format(x.shape))
    logger.debug("v shape: {}".format(v.shape))
    
    # ----------------------------------------------------------------
    # compute new tensor
    
    # X' = V * X * transpose(V)'
    # https://utsv.net/solid-mechanics/introduction/2nd-order-tensor-transformations
    # https://link.springer.com/content/pdf/bbm:978-3-662-04168-0/1.pdf
    
    logger.debug("Computing new tensor")
    
    # compute proper transpose
    v_ = np.transpose(v,axes=(0,2,1))
        
    # compute new tensor value
    new_x = np.matmul(v, x)
    new_x = np.matmul(new_x, v_)
    
    # log new information
    logger.debug("new_x shape: {}".format(new_x.shape))
    logger.debug("new_x: {}".format(new_x))
    
    return new_x

## test util functions

Test shape consistency

In [485]:
a = np.random.random((16992, 6))
# ensure tensor is a sym
# -> if it is a vector, it will return a single tensor
# -> if it is an array of vectors, it will return an array of tensors
a = to_sym_tensor(a)
print(a.shape)
b = np.random.random((16992, 3))
print(b.shape)
a_new = apply_tensor_transformation(a, b, log_level=logging.WARN)
print(a_new.shape)

(16992, 3, 3)
(16992, 3)
(16992, 1, 1)


In [525]:
def manual_test(ma,mb):
    assert len(ma) == len(mb), "ma must be the same length as mb: {}, {}".format(ma.shape,mb.shape)
    xs = []
    for a,b in zip(ma,mb):
        x = np.matmul(np.matmul(b, a), b.T)
        xs.append(x)
    return xs

Test numerical value

In [507]:
a = [
    [1,1,1,0,0,0],
    [2,1,2,2,4,2],
]
b = np.asarray([
        [2,0,0],
        [0,1,1]
    ])

a = to_sym_tensor(a)

a_true = manual_test(a,b)
a_est = np.squeeze(apply_tensor_transformation(a, b))

np.all(a_true == a_est)

True

In [522]:
a = np.random.random((5,6))
b = np.random.random((3))

a = to_sym_tensor(a)
b_test = np.repeat(np.expand_dims(b, 0), a.shape[0], axis=0)

a_true = manual_test(a,b_test)
a_est = np.squeeze(apply_tensor_transformation(a, b))

np.all(a_true == a_est)

(5, 3, 3) (5, 3)


True

In [531]:
a = np.random.random((5,6))
b = np.random.random((5,3))

a = to_sym_tensor(a)
a_true = manual_test(a,b)
a_est = np.squeeze(apply_tensor_transformation(a, b))

np.all(a_true == a_est)

True

In [534]:
a = np.random.random((5,6))
b = np.random.random((5,3,3))

a = to_sym_tensor(a)
a_true = manual_test(a,b)
a_est = np.squeeze(apply_tensor_transformation(a, b))

np.all(a_true == a_est)

True

test computation efficiency

In [549]:
import time

a = np.random.random((1000000, 6))
a = to_sym_tensor(a)
b = np.random.random((1000000, 3))

start_time = time.time()
apply_tensor_transformation(a, b)
t1 = time.time() - start_time
print("custom code: {} s".format(t1))
start_time = time.time()
manual_test(a, b)
t2 = time.time() - start_time
print("simple code: {} s".format(t2))


print("efficiency: {}".format(t2/t1))

custom code: 0.0784604549407959 s
simple code: 2.4308252334594727 s
efficiency: 30.98153375855017


# Compute stress in FF direction

In [565]:
from pathlib import Path
import numpy as np
import logging

import pyvista as pv
pv.set_jupyter_backend("pythreejs")

from project_heart.enums import *
from project_heart.lv import LV

In [566]:
directory = Path("C:/Users/igorp/OneDrive - University of South Florida/Igor/Documents/Paper 1/stress_strain_plots/ref_files")
filename = "57.32_40.94_10.08.2022_LVRNN_IDEAL_V4"
# filename = "62.71_68.65_10.08.2022_LVRNN_TYPEA_V4"

In [567]:
from project_heart.examples import get_lv_ideal, get_lv_typeA
lv = get_lv_ideal(str(directory/filename) + ".xplt", statesfile=None)
# lv = get_lv_typeA(str(directory/filename) + ".xplt", statesfile=None)

In [568]:
fibers = np.asarray(lv.states.data["fiber vector"])
stress = np.asarray(lv.states.data["stress"])
strain = np.asarray(lv.states.data["Lagrange strain"])

In [569]:
fibers.shape, stress.shape, strain.shape

((163, 16992, 3), (163, 16992, 6), (163, 16992, 6))

In [570]:
stress_T = []
for s, f in zip(stress, fibers):
    s = to_sym_tensor(s)
    s = apply_tensor_transformation(s, f)
    stress_T.append(s)
stress_T = np.asarray(stress_T)

In [571]:
strain_T = []
for s, f in zip(strain, fibers):
    s = to_sym_tensor(s)
    s = apply_tensor_transformation(s, f)
    strain_T.append(s)
strain_T = np.asarray(strain_T)

In [572]:
stress_T = np.squeeze(stress_T, -1)
stress_T_stack = np.hstack(stress_T)

In [573]:
strain_T = np.squeeze(strain_T, -1)
strain_T_stack = np.hstack(strain_T)

In [574]:
idx = np.arange(1,stress_T_stack.shape[0]+1).reshape(-1,1)

In [575]:
np.savetxt("stress_FF.csv", np.hstack((idx, stress_T_stack)), delimiter=",", fmt="%.3f")

In [576]:
np.savetxt("strain_FF.csv", np.hstack((idx, strain_T_stack)), delimiter=",", fmt="%.3f")